In [154]:
!pip install pandasql
from pandasql import sqldf
import pandas as pd

pysqldf = lambda q: sqldf(q, globals())

In [282]:
# readmissions = pd.read_csv('/Users/athenacasarotto/ds4a_data/readmissions.csv')

readmissions = pd.read_csv('readmissions.csv')
hospitals = pd.read_csv('hospitals.csv')
footnotes = pd.read_csv('footnotes.csv')
sdoh = pd.read_csv('sdoh.csv')
payments = pd.read_csv('payments.csv')
patient_mix = pd.read_csv('patient_mix.csv')
pmr_2018 = pd.read_csv('Medicare_Inpatient_Hospitals_by_Provider_2018.csv')
pmr_2019 = pd.read_csv('Medicare_Inpatient_Hospitals_by_Provider_2019.csv')
pmr_2020 = pd.read_csv('Medicare_Inpatient_Hospitals_by_Provider_2020.csv')
pmr_2021 = pd.read_csv('Medicare_Inpatient_Hospitals_by_Provider_2021.csv')
pmr_agg = pd.read_csv('pmr_agg.csv')
sdoh_2020 = pd.read_csv('SVI_2020_US_county.csv')
sdoh_2018 = pd.read_csv('SVI_2018_US_county.csv')

In [156]:
payments.head()

,facility_id,facility_name,address,city,state,zip_code,county_name,phone_number,payment_measure_id,payment_measure_name,...,payment,lower_estimate,higher_estimate,payment_footnote,value_of_care_display_id,value_of_care_display_name,value_of_care_category,value_of_care_footnote,start_date,end_date
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$26,894","$24,890","$29,027",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,7/1/18,6/30/21
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,835","$16,947","$18,783",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Better Mortality and Average Payment,NaN,7/1/18,6/30/21
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,182","$19,430","$20,653",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,NaN,7/1/18,6/30/21
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$21,623","$20,104","$23,309",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,4/1/18,3/31/21
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$25,814","$22,668","$29,475",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,7/1/18,6/30/21


In [233]:
a = """
select 2018 as year, * from pmr_2018

union all

select 2019 as year, * from pmr_2019

union all

select 2020 as year, * from pmr_2020

union all

select 2021 as year, * from pmr_2021
"""

sdoh_rename = """
select 2020 as year, 
  st as state_id,
  state as state,
  upper(st_abbr) as state_abbr,
  upper(county) as county,
  fips as fips,
  location as location,
  area_sqmi as area_sqmi,
  e_totpop as population,
  m_totpop as population_moe,
  e_hu as housing_units,
  m_hu as housing_units_moe,
  e_hh as households,
  m_hh as households_moe,
  e_daypop as daytime_population,
  
  -- socioeconomic status (green)
  e_pov150 as below_150_poverty,
  m_pov150 as below_150_poverty_moe,
  ep_pov150 as below_150_poverty_pct,
  mp_pov150 as below_150_poverty_pct_moe,
  epl_pov150 as below_150_poverty_percentile,
  f_pov150 as below_150_poverty_is_90_percentile,
  e_unemp as adults_unemployed,
  m_unemp as adults_unemployed_moe,
  ep_unemp as unemployment_rate,
  mp_unemp as unemployment_rate_moe,
  epl_unemp as unemployment_percentile,
  f_unemp as unemployment_is_90_percentile,
  e_hburd as housing_cost_burdened,
  m_hburd as housing_cost_burdened_moe,
  ep_hburd as housing_cost_burdened_pct,
  mp_hburd as housing_cost_burdened_pct_moe,
  epl_hburd as housing_cost_burdened_percentile,
  f_hburd as housing_cost_burdened_is_90_percentile,
  e_nohsdp as no_hs_diploma,
  m_nohsdp as no_hs_diploma_moe,
  ep_nohsdp as no_hs_diploma_pct,
  mp_nohsdp as no_hs_diploma_pct_moe,
  epl_nohsdp as no_hs_diploma_percentile,
  f_nohsdp as no_hs_diploma_is_90_percentile,
  e_uninsur as uninsured,
  m_uninsur as uninsured_moe,
  ep_uninsur as uninsured_pct,
  mp_uninsur as uninsured_pct_moe,
  epl_uninsur as uninsured_percentile,
  f_uninsur as uninsured_is_90_percentile,
  spl_theme1 as socioeconomic_status_sum,
  rpl_theme1 as socioeconomic_status_percentile_rank,
  f_theme1 as socioeconomic_status_90_percentile_count,
  
  -- household characteristics (orange)
  e_age65 as aged_65_over,
  m_age65 as aged_65_over_moe,
  ep_age65 as aged_65_over_pct,
  mp_age65 as aged_65_over_pct_moe,
  epl_age65 as aged_65_over_percentile,
  f_age65 as aged_65_over_is_90_percentile,
  e_age17 as aged_17_under,
  m_age17 as aged_17_under_moe,
  ep_age17 as aged_17_under_pct,
  mp_age17 as aged_17_under_pct_moe,
  epl_age17 as aged_17_under_percentile,
  f_age17 as aged_17_under_is_90_percentile,
  e_disabl as disabled,
  m_disabl as disabled_moe,
  ep_disabl as disabled_pct,
  mp_disabl as disabled_pct_moe,
  epl_disabl as disabled_percentile,
  f_disabl as disabled_is_90_percentile,
  e_sngpnt as single_parent_hh,
  m_sngpnt as single_parent_hh_moe,
  ep_sngpnt as single_parent_hh_pct,
  mp_sngpnt as single_parent_hh_pct_moe,
  epl_sngpnt as single_parent_hh_percentile,
  f_sngpnt as single_parent_hh_is_90_percentile,
  e_limeng as limited_english,
  m_limeng as limited_english_moe,
  ep_limeng as limited_english_pct,
  mp_limeng as limited_english_pct_moe,
  epl_limeng as limited_english_percentile,
  f_limeng as limited_english_is_90_percentile,
  spl_theme2 as hh_characteristics_sum,
  rpl_theme2 as hh_characteristics_percentile_rank,
  f_theme2 as hh_characteristics_90_percentile_count,
  
  -- racial & ethnic minority status (purple)
  e_minrty as minorities,
  m_minrty as minorities_moe,
  ep_minrty as minorities_pct,
  mp_minrty as minorities_pct_moe,
  epl_minrty as minorities_percentile,
  f_minrty as minorities_is_90_percentile,
  spl_theme3 as minority_status_sum,
  rpl_theme3 as minority_status_percentile_rank,
  f_theme3 as minority_status_90_percentile_count,
  e_afam as african_american,
  m_afam as african_american_moe,
  ep_afam as african_american_pct,
  mp_afam as african_american_pct_moe,
  e_hisp as hispanic,
  m_hisp as hispanic_moe,
  ep_hisp as hispanic_pct,
  mp_hisp as hispanic_pct_moe,
  e_asian as asian,
  m_asian as asian_moe,
  ep_asian as asian_pct,
  mp_asian as asian_pct_moe,
  e_aian as american_indian,
  m_aian as american_indian_moe,
  ep_aian as american_indian_pct,
  mp_aian as american_indian_pct_moe,
  e_nhpi as native_hawaiian,
  m_nhpi as native_hawaiian_moe,
  ep_nhpi as native_hawaiian_pct,
  mp_nhpi as native_hawaiian_pct_moe,
  e_twomore as two_or_more_races,
  m_twomore as two_or_more_races_moe,
  ep_twomore as two_or_more_races_pct,
  mp_twomore as two_or_more_races_pct_moe,
  e_otherrace as other_race,
  m_otherrace as other_race_moe,
  ep_otherrace as other_race_pct,
  mp_otherrace as other_race_pct_moe,
  
  -- housing type/transportation (blue)
  e_munit as apartments,
  m_munit as apartments_moe,
  ep_munit as apartments_pct,
  mp_munit as apartments_pct_moe,
  epl_munit as apartments_percentile,
  f_munit as apartments_is_90_percentile,
  e_mobile as mobile_homes,
  m_mobile as mobile_homes_moe,
  ep_mobile as mobile_homes_pct,
  mp_mobile as mobile_homes_pct_moe,
  epl_mobile as mobile_homes_percentile,
  f_mobile as mobile_homes_is_90_percentile,
  e_crowd as crowded_housing,
  m_crowd as crowded_housing_moe,
  ep_crowd as crowded_housing_pct,
  mp_crowd as crowded_housing_pct_moe,
  epl_crowd as crowded_housing_percentile,
  f_crowd as crowded_housing_is_90_percentile,
  e_noveh as no_vehicle,
  m_noveh as no_vehicle_moe,
  ep_noveh as no_vehicle_pct,
  mp_noveh as no_vehicle_pct_moe,
  epl_noveh as no_vehicle_percentile,
  f_noveh as no_vehicle_is_90_percentile,
  e_groupq as group_quarters,
  m_groupq as group_quarters_moe,
  ep_groupq as group_quarters_pct,
  mp_groupq as group_quarters_pct_moe,
  epl_groupq as group_quarters_percentile,
  f_groupq as group_quarters_is_90_percentile,
  spl_theme4 as housing_status_sum,
  rpl_theme4 as housing_status_percentile_rank,
  f_theme4 as housing_status_90_percentile_count,
  e_noint as hh_without_internet,
  m_noint as hh_without_internet_moe,
  ep_noint as hh_without_internet_pct,
  mp_noint as hh_without_internet_pct_moe,
  
  spl_themes as series_themes_sum,
  rpl_themes as overall_percentile_ranking,
  f_total as theme_90_percentile_count
from sdoh_2020
"""

sdoh_cols = """
select
  s.year, 
  s.state_id,
  s.state,
  s.state_abbr,
  s.county,
  s.fips,
  s.location,
  s.area_sqmi,
  s.population,
  s.population_moe,
  s.housing_units,
  s.housing_units_moe,
  s.households,
  s.households_moe,
  s.daytime_population,
  
  -- socioeconomic status (green)
  s.below_150_poverty,
  s.below_150_poverty_moe,
  s.below_150_poverty_pct,
  s.below_150_poverty_pct_moe,
  s.below_150_poverty_percentile,
  s.below_150_poverty_is_90_percentile,
  s.adults_unemployed,
  s.adults_unemployed_moe,
  s.unemployment_rate,
  s.unemployment_rate_moe,
  s.unemployment_percentile,
  s.unemployment_is_90_percentile,
  s.housing_cost_burdened,
  s.housing_cost_burdened_moe,
  s.housing_cost_burdened_pct,
  s.housing_cost_burdened_pct_moe,
  s.housing_cost_burdened_percentile,
  s.housing_cost_burdened_is_90_percentile,
  s.no_hs_diploma,
  s.no_hs_diploma_moe,
  s.no_hs_diploma_pct,
  s.no_hs_diploma_pct_moe,
  s.no_hs_diploma_percentile,
  s.no_hs_diploma_is_90_percentile,
  s.uninsured,
  s.uninsured_moe,
  s.uninsured_pct,
  s.uninsured_pct_moe,
  s.uninsured_percentile,
  s.uninsured_is_90_percentile,
  s.socioeconomic_status_sum,
  s.socioeconomic_status_percentile_rank,
  s.socioeconomic_status_90_percentile_count,
  
  -- household characteristics (orange)
  s.aged_65_over,
  s.aged_65_over_moe,
  s.aged_65_over_pct,
  s.aged_65_over_pct_moe,
  s.aged_65_over_percentile,
  s.aged_65_over_is_90_percentile,
  s.aged_17_under,
  s.aged_17_under_moe,
  s.aged_17_under_pct,
  s.aged_17_under_pct_moe,
  s.aged_17_under_percentile,
  s.aged_17_under_is_90_percentile,
  s.disabled,
  s.disabled_moe,
  s.disabled_pct,
  s.disabled_pct_moe,
  s.disabled_percentile,
  s.disabled_is_90_percentile,
  s.single_parent_hh,
  s.single_parent_hh_moe,
  s.single_parent_hh_pct,
  s.single_parent_hh_pct_moe,
  s.single_parent_hh_percentile,
  s.single_parent_hh_is_90_percentile,
  s.limited_english,
  s.limited_english_moe,
  s.limited_english_pct,
  s.limited_english_pct_moe,
  s.limited_english_percentile,
  s.limited_english_is_90_percentile,
  s.hh_characteristics_sum,
  s.hh_characteristics_percentile_rank,
  s.hh_characteristics_90_percentile_count,
  
  -- racial & ethnic minority status (purple)
  s.minorities,
  s.minorities_moe,
  s.minorities_pct,
  s.minorities_pct_moe,
  s.minorities_percentile,
  s.minorities_is_90_percentile,
  s.minority_status_sum,
  s.minority_status_percentile_rank,
  s.minority_status_90_percentile_count,
  s.african_american,
  s.african_american_moe,
  s.african_american_pct,
  s.african_american_pct_moe,
  s.hispanic,
  s.hispanic_moe,
  s.hispanic_pct,
  s.hispanic_pct_moe,
  s.asian,
  s.asian_moe,
  s.asian_pct,
  s.asian_pct_moe,
  s.american_indian,
  s.american_indian_moe,
  s.american_indian_pct,
  s.american_indian_pct_moe,
  s.native_hawaiian,
  s.native_hawaiian_moe,
  s.native_hawaiian_pct,
  s.native_hawaiian_pct_moe,
  s.two_or_more_races,
  s.two_or_more_races_moe,
  s.two_or_more_races_pct,
  s.two_or_more_races_pct_moe,
  s.other_race,
  s.other_race_moe,
  s.other_race_pct,
  s.other_race_pct_moe,
  
  -- housing type/transportation (blue)
  s.apartments,
  s.apartments_moe,
  s.apartments_pct,
  s.apartments_pct_moe,
  s.apartments_percentile,
  s.apartments_is_90_percentile,
  s.mobile_homes,
  s.mobile_homes_moe,
  s.mobile_homes_pct,
  s.mobile_homes_pct_moe,
  s.mobile_homes_percentile,
  s.mobile_homes_is_90_percentile,
  s.crowded_housing,
  s.crowded_housing_moe,
  s.crowded_housing_pct,
  s.crowded_housing_pct_moe,
  s.crowded_housing_percentile,
  s.crowded_housing_is_90_percentile,
  s.no_vehicle,
  s.no_vehicle_moe,
  s.no_vehicle_pct,
  s.no_vehicle_pct_moe,
  s.no_vehicle_percentile,
  s.no_vehicle_is_90_percentile,
  s.group_quarters,
  s.group_quarters_moe,
  s.group_quarters_pct,
  s.group_quarters_pct_moe,
  s.group_quarters_percentile,
  s.group_quarters_is_90_percentile,
  s.housing_status_sum,
  s.housing_status_percentile_rank,
  s.housing_status_90_percentile_count,
  s.hh_without_internet,
  s.hh_without_internet_moe,
  s.hh_without_internet_pct,
  s.hh_without_internet_pct_moe,
  
  s.series_themes_sum,
  s.overall_percentile_ranking,
  s.theme_90_percentile_count
from sdoh
"""


b = pysqldf(sdoh_rename)
b
# b.to_csv(r'sdoh.csv', index=False)

In [239]:
pmr = """
select
  year,
  facility_id,
  facility_name,
  address,
  city,
  zip_code,
  state,
  Tot_Submtd_Cvrd_Chrg as total_covered_charges,
  Tot_Pymt_Amt as total_payment_amount,
  Tot_Mdcr_Pymt_Amt as total_medicare_payment_amount,
  Tot_Dschrgs as total_discharges,
  Tot_Benes as total_medicare_beneficiaries,
  Tot_Cvrd_Days as total_covered_days,
  Tot_Days as total_days,
  Bene_Avg_Age as beneficiary_avg_age,
  Bene_Age_LT_65_Cnt as number_beneficiaries_under_65,
  Bene_Age_65_74_Cnt as number_beneficiaries_65_to_74,
  Bene_Age_75_84_Cnt as number_beneficiaries_75_to_84,
  Bene_Age_Gt_84_Cnt as number_beneficiaries_85_over,
  Bene_Feml_Cnt as number_female_beneficiaries,
  Bene_Male_Cnt as number_male_beneficiaries,
  Bene_Race_Wht_Cnt as number_white_beneficiaries,
  Bene_Race_Black_Cnt as number_black_beneficiaries,
  Bene_Race_API_Cnt as number_asian_pi_beneficiaries,
  Bene_Race_Hspnc_Cnt as number_hispanic_beneficiaries,
  Bene_Race_Natind_Cnt as number_american_indian_beneficiaries,
  Bene_Race_Othr_Cnt as number_other_race_beneficiaries,
  Bene_Dual_Cnt as number_beneficiaries_medicare_medicaid,
  Bene_Ndual_Cnt as number_beneficiaries_medicare_only,
  Bene_CC_AF_Pct as pct_beneficiaries_afib,
  Bene_CC_Alzhmr_Pct as pct_beneficiaries_alzheimers,
  Bene_CC_Asthma_Pct as pct_beneficiaries_asthma,
  Bene_CC_Cncr_Pct as pct_beneficiaries_cancer,
  Bene_CC_CHF_Pct as pct_beneficiaries_congestive_heart_failure,
  Bene_CC_CKD_Pct as pct_beneficiaries_chronic_kidney_disease,
  Bene_CC_COPD_Pct as pct_beneficiaries_copd,
  Bene_CC_Dprssn_Pct as pct_beneficiaries_depression,
  Bene_CC_Dbts_Pct as pct_beneficiaries_diabetes,
  Bene_CC_Hyplpdma_Pct as pct_beneficiaries_hyperlipidemia,
  Bene_CC_Hyprtnsn_Pct as pct_beneficiaries_hypertension,
  Bene_CC_IHD_Pct as pct_beneficiaries_ischemic_heart_disease,
  Bene_CC_Opo_Pct as pct_beneficiaries_osteoporosis,
  Bene_CC_RAOA_Pct as pct_beneficiaries_arthritis,
  Bene_CC_Sz_Pct as pct_beneficiaries_psychotic_disorders,
  Bene_CC_Strok_Pct as pct_beneficiaries_stroke,
  Bene_Avg_Risk_Scre as average_hcc_beneficiary_risk_score
from patient_mix
"""

b = pysqldf(pmr)
b
# b.to_csv(r'pmr.csv', index=False)

In [249]:
#4-year sums of beneficiaries are misleading - many patients persist over the years, so avoid using those if possible

pmr_agg = """
select
  facility_id,
  sum(total_covered_charges) as total_covered_charges,
  avg(total_covered_charges) as avg_covered_charges_py,
  sum(total_payment_amount) as total_payment_amount,
  avg(total_payment_amount) as avg_payment_amount_py,
  sum(total_medicare_payment_amount) as total_medicare_payment_amount,
  avg(total_medicare_payment_amount) as avg_medicare_payment_amount_py,
  sum(total_discharges) as total_discharges,
  avg(total_discharges) as avg_discharges_py,
  sum(total_medicare_beneficiaries) as total_medicare_beneficiaries,
  avg(total_medicare_beneficiaries) as avg_medicare_beneficiaries_py,
  sum(total_covered_days) as total_covered_days,
  avg(total_covered_days) as avg_covered_days_py,
  sum(total_days) as total_days,
  avg(total_days) as avg_days_py,
  
  sum(beneficiary_avg_age * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as beneficiary_avg_age,
  sum(number_beneficiaries_under_65) as number_beneficiaries_under_65,
  (sum(number_beneficiaries_under_65) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_beneficiaries_under_65_py,
  sum(number_beneficiaries_65_to_74) as number_beneficiaries_65_to_74,
  (sum(number_beneficiaries_65_to_74) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_beneficiaries_65_to_74_py,
  sum(number_beneficiaries_75_to_84) as number_beneficiaries_75_to_84,
  (sum(number_beneficiaries_75_to_84) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_beneficiaries_75_to_84_py,
  sum(number_beneficiaries_85_over) as number_beneficiaries_85_over,
  (sum(number_beneficiaries_85_over) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_beneficiaries_85_over_py,
  sum(number_female_beneficiaries) as number_female_beneficiaries,
  (sum(number_female_beneficiaries) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_female_beneficiaries_py,
  sum(number_male_beneficiaries) as number_male_beneficiaries,
  (sum(number_male_beneficiaries) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_male_beneficiaries_py,
  sum(number_white_beneficiaries) as number_white_beneficiaries,
  (sum(number_white_beneficiaries) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_white_beneficiaries_py,
  sum(number_black_beneficiaries) as number_black_beneficiaries,
  (sum(number_black_beneficiaries) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_black_beneficiaries_py,
  sum(number_asian_pi_beneficiaries) as number_asian_pi_beneficiaries,
  (sum(number_asian_pi_beneficiaries) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_asian_pi_beneficiaries_py,
  sum(number_hispanic_beneficiaries) as number_hispanic_beneficiaries,
  (sum(number_hispanic_beneficiaries) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_hispanic_beneficiaries_py,
  sum(number_american_indian_beneficiaries) as number_american_indian_beneficiaries,
  (sum(number_american_indian_beneficiaries) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_american_indian_beneficiaries_py,
  sum(number_other_race_beneficiaries) as number_other_race_beneficiaries,
  (sum(number_other_race_beneficiaries) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_other_race_beneficiaries_py,
  sum(number_beneficiaries_medicare_medicaid) as number_beneficiaries_medicare_medicaid,
  (sum(number_beneficiaries_medicare_medicaid) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_beneficiaries_medicare_medicaid_py,
  sum(number_beneficiaries_medicare_only) as number_beneficiaries_medicare_only,
  (sum(number_beneficiaries_medicare_only) / sum(total_medicare_beneficiaries)) * avg(total_medicare_beneficiaries) as avg_number_beneficiaries_medicare_only_py,
  
  sum(pct_beneficiaries_afib * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_afib,
  sum(pct_beneficiaries_alzheimers * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_alzheimers,
  sum(pct_beneficiaries_cancer * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_cancer,
  sum(pct_beneficiaries_congestive_heart_failure * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_congestive_heart_failure,
  sum(pct_beneficiaries_chronic_kidney_disease * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_chronic_kidney_disease,
  sum(pct_beneficiaries_copd * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_copd,
  sum(pct_beneficiaries_depression * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_depression,
  sum(pct_beneficiaries_diabetes * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_diabetes,
  sum(pct_beneficiaries_hyperlipidemia * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_hyperlipidemia,
  sum(pct_beneficiaries_hypertension * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_hypertension,
  sum(pct_beneficiaries_ischemic_heart_disease * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_ischemic_heart_disease,
  sum(pct_beneficiaries_osteoporosis * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_osteoporosis,
  sum(pct_beneficiaries_arthritis * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_arthritis,
  sum(pct_beneficiaries_psychotic_disorders * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_psychotic_disorders,
  sum(pct_beneficiaries_stroke * total_medicare_beneficiaries) / sum(total_medicare_beneficiaries) as pct_beneficiaries_stroke,
  sum(average_hcc_beneficiary_risk_score * coalesce(total_medicare_beneficiaries, 1)) / sum(coalesce(total_medicare_beneficiaries,1)) as average_hcc_beneficiary_risk_score,
  avg(average_hcc_beneficiary_risk_score) as avg_hcc_beneficiary_risk_score
from patient_mix
group by 1
"""

pmr_cols = """
select
  m.facility_id,
  m.total_covered_charges,
  m.avg_covered_charges_py,
  m.total_payment_amount,
  m.avg_payment_amount_py,
  m.total_medicare_payment_amount,
  m.avg_medicare_payment_amount_py,
  m.total_discharges,
  m.avg_discharges_py,
  m.total_medicare_beneficiaries,
  m.avg_medicare_beneficiaries_py,
  m.total_covered_days,
  m.avg_covered_days_py,
  m.total_days,
  m.avg_days_py,
  
  m.beneficiary_avg_age,
  m.number_beneficiaries_under_65,
  m.avg_number_beneficiaries_under_65_py,
  m.number_beneficiaries_65_to_74,
  m.avg_number_beneficiaries_65_to_74_py,
  m.number_beneficiaries_75_to_84,
  m.avg_number_beneficiaries_75_to_84_py,
  m.number_beneficiaries_85_over,
  m.avg_number_beneficiaries_85_over_py,
  m.number_female_beneficiaries,
  m.avg_number_female_beneficiaries_py,
  m.number_male_beneficiaries,
  m.avg_number_male_beneficiaries_py,
  m.number_white_beneficiaries,
  m.avg_number_white_beneficiaries_py,
  m.number_black_beneficiaries,
  m.avg_number_black_beneficiaries_py,
  m.number_asian_pi_beneficiaries,
  m.avg_number_asian_pi_beneficiaries_py,
  m.number_hispanic_beneficiaries,
  m.avg_number_hispanic_beneficiaries_py,
  m.number_american_indian_beneficiaries,
  m.avg_number_american_indian_beneficiaries_py,
  m.number_other_race_beneficiaries,
  m.avg_number_other_race_beneficiaries_py,
  m.number_beneficiaries_medicare_medicaid,
  m.avg_number_beneficiaries_medicare_medicaid_py,
  m.number_beneficiaries_medicare_only,
  m.avg_number_beneficiaries_medicare_only_py,
  
  m.pct_beneficiaries_afib,
  m.pct_beneficiaries_alzheimers,
  m.pct_beneficiaries_cancer,
  m.pct_beneficiaries_congestive_heart_failure,
  m.pct_beneficiaries_chronic_kidney_disease,
  m.pct_beneficiaries_copd,
  m.pct_beneficiaries_depression,
  m.pct_beneficiaries_diabetes,
  m.pct_beneficiaries_hyperlipidemia,
  m.pct_beneficiaries_hypertension,
  m.pct_beneficiaries_ischemic_heart_disease,
  m.pct_beneficiaries_osteoporosis,
  m.pct_beneficiaries_arthritis,
  m.pct_beneficiaries_psychotic_disorders,
  m.pct_beneficiaries_stroke,
  m.average_hcc_beneficiary_risk_score,
  m.avg_hcc_beneficiary_risk_score
from pmr_agg m
"""

b = pysqldf(pmr_agg)
b
b.to_csv(r'pmr_agg.csv', index=False)

In [281]:
r = """
select
  r.facility_name,
  r.facility_id,
  upper(r.state) as state,
  replace(substring(r.measure_name, 10), '-HRRP', '') as condition,
  case 
    when replace(substring(r.measure_name, 10), '-HRRP', '') = 'HIP-KNEE' then 'Total Hip/Knee Arthroplasty'
    when replace(substring(r.measure_name, 10), '-HRRP', '') = 'COPD' then 'Chronic Obstructive Pulmonary Disease'
    when replace(substring(r.measure_name, 10), '-HRRP', '') = 'AMI' then 'Acute Myocardial Infarction'
    when replace(substring(r.measure_name, 10), '-HRRP', '') = 'PN' then 'Pneumonia'
    when replace(substring(r.measure_name, 10), '-HRRP', '') = 'HF' then 'Heart Failure'
    when replace(substring(r.measure_name, 10), '-HRRP', '') = 'CABG' then 'Coronary Artery Bypass Graft'
    end as condition_name,
  r.measure_name,
  r.number_of_discharges,
  f_r.footnote_text as readmissions_footnote,
  r.excess_readmission_ratio,
  r.predicted_readmission_rate,
  r.expected_readmission_rate,
  r.predicted_readmission_rate / nullif(r.expected_readmission_rate, 0) as calc_err,
  round(r.predicted_readmission_rate / nullif(r.expected_readmission_rate, 0), 4) = r.excess_readmission_ratio as calc_err_match,
  cast(nullif(r.number_of_readmissions, 'Too Few to Report') as int) as number_of_readmissions,
  r.start_date as readm_start_date,
  r.end_date as readm_end_date
from readmissions r 
left join footnotes f_r
  on r.footnote = f_r.footnote_id
"""

b = pysqldf(r)
b
# b.to_csv(r'readmissions.csv', index=False)

te = timely and effective care

In [286]:
q = """
select
  coalesce(r.facility_name, p.facility_name) as facility_name,
  coalesce(r.facility_id, p.facility_id) as facility_id,
  upper(coalesce(h.county_name, p.county_name)) as county_name,
  upper(coalesce(r.state, h.state, p.state)) as state,
  coalesce(replace(substring(r.measure_name, 10), '-HRRP', ''), substring(replace(p.payment_measure_id, '_', '-'), 9)) as condition,
  coalesce(r.condition_name, case 
    when substring(replace(p.payment_measure_id, '_', '-'), 9) = 'HIP-KNEE' then 'Total Hip/Knee Arthroplasty'
    when substring(replace(p.payment_measure_id, '_', '-'), 9) = 'COPD' then 'Chronic Obstructive Pulmonary Disease'
    when substring(replace(p.payment_measure_id, '_', '-'), 9) = 'AMI' then 'Acute Myocardial Infarction'
    when substring(replace(p.payment_measure_id, '_', '-'), 9) = 'PN' then 'Pneumonia'
    when substring(replace(p.payment_measure_id, '_', '-'), 9) = 'HF' then 'Heart Failure'
    when substring(replace(p.payment_measure_id, '_', '-'), 9) = 'CABG' then 'Coronary Artery Bypass Graft'
    end) as condition_name,
  r.measure_name,
  p.payment_measure_id,
  p.payment_measure_name,
  r.number_of_discharges,
  r.readmissions_footnote,
  r.excess_readmission_ratio,
  r.predicted_readmission_rate,
  r.expected_readmission_rate,
  r.calc_err,
  r.calc_err_match,
  r.number_of_readmissions,
  r.readm_start_date,
  r.readm_end_date,
  p.start_date as payments_start_date,
  p.end_date as payments_end_date,
  coalesce(h.address, p.address) as address,
  coalesce(h.city, p.city) as city,
  coalesce(h.zip_code, p.zip_code) as zip_code,
  h.hospital_type,
  h.hospital_ownership,
  coalesce(h.emergency_services, 'No') = 'Yes' as emergency_services,
  coalesce(h.meets_criteria_for_promoting_interoperability_of_ehrs, 'N') = 'Y' as meets_criteria_for_promoting_interoperability_of_ehrs,
  h.hospital_overall_rating,
  f_h_rating.footnote_text as hospital_overall_rating_footnote,
  h.mort_group_measure_count,
  h.count_of_facility_mort_measures,
  h.count_of_mort_measures_better,
  h.count_of_mort_measures_no_different,
  h.count_of_mort_measures_worse,
  f_h_mort.footnote_text as mort_group_footnote,
  h.safety_group_measure_count,
  h.count_of_facility_safety_measures,
  h.count_of_safety_measures_better,
  h.count_of_safety_measures_no_different,
  h.count_of_safety_measures_worse,
  f_h_safety.footnote_text as safety_group_footnote,
  h.readm_group_measure_count,
  h.count_of_facility_readm_measures,
  h.count_of_readm_measures_better,
  h.count_of_readm_measures_no_different,
  h.count_of_readm_measures_worse,
  f_h_readm.footnote_text as readm_group_footnote,
  h.pt_exp_group_measure_count,
  h.count_of_facility_pt_exp_measures,
  f_h_pt_exp.footnote_text as pt_exp_group_footnote,
  h.te_group_measure_count,
  h.count_of_facility_te_measures,
  f_h_te.footnote_text as te_group_footnote,
  p.payment_category,
  p.denominator,
  p.payment,
  p.lower_estimate,
  p.higher_estimate,
  f_p.footnote_text as payment_footnote,
  p.value_of_care_display_id,
  p.value_of_care_display_name,
  p.value_of_care_category,
  f_p_vc.footnote_text as value_of_care_footnote,
  
  m.total_covered_charges,
  m.avg_covered_charges_py,
  m.total_payment_amount,
  m.avg_payment_amount_py,
  m.total_medicare_payment_amount,
  m.avg_medicare_payment_amount_py,
  m.total_discharges,
  m.avg_discharges_py,
  m.total_medicare_beneficiaries,
  m.avg_medicare_beneficiaries_py,
  m.total_covered_days,
  m.avg_covered_days_py,
  m.total_days,
  m.avg_days_py,
  
  m.beneficiary_avg_age,
  m.number_beneficiaries_under_65,
  m.avg_number_beneficiaries_under_65_py,
  m.number_beneficiaries_65_to_74,
  m.avg_number_beneficiaries_65_to_74_py,
  m.number_beneficiaries_75_to_84,
  m.avg_number_beneficiaries_75_to_84_py,
  m.number_beneficiaries_85_over,
  m.avg_number_beneficiaries_85_over_py,
  m.number_female_beneficiaries,
  m.avg_number_female_beneficiaries_py,
  m.number_male_beneficiaries,
  m.avg_number_male_beneficiaries_py,
  m.number_white_beneficiaries,
  m.avg_number_white_beneficiaries_py,
  m.number_black_beneficiaries,
  m.avg_number_black_beneficiaries_py,
  m.number_asian_pi_beneficiaries,
  m.avg_number_asian_pi_beneficiaries_py,
  m.number_hispanic_beneficiaries,
  m.avg_number_hispanic_beneficiaries_py,
  m.number_american_indian_beneficiaries,
  m.avg_number_american_indian_beneficiaries_py,
  m.number_other_race_beneficiaries,
  m.avg_number_other_race_beneficiaries_py,
  m.number_beneficiaries_medicare_medicaid,
  m.avg_number_beneficiaries_medicare_medicaid_py,
  m.number_beneficiaries_medicare_only,
  m.avg_number_beneficiaries_medicare_only_py,
  
  m.pct_beneficiaries_afib,
  m.pct_beneficiaries_alzheimers,
  m.pct_beneficiaries_cancer,
  m.pct_beneficiaries_congestive_heart_failure,
  m.pct_beneficiaries_chronic_kidney_disease,
  m.pct_beneficiaries_copd,
  m.pct_beneficiaries_depression,
  m.pct_beneficiaries_diabetes,
  m.pct_beneficiaries_hyperlipidemia,
  m.pct_beneficiaries_hypertension,
  m.pct_beneficiaries_ischemic_heart_disease,
  m.pct_beneficiaries_osteoporosis,
  m.pct_beneficiaries_arthritis,
  m.pct_beneficiaries_psychotic_disorders,
  m.pct_beneficiaries_stroke,
  m.average_hcc_beneficiary_risk_score,
  m.avg_hcc_beneficiary_risk_score,
  
  s.fips,
  s.location,
  s.area_sqmi,
  s.population,
  s.population_moe,
  s.housing_units,
  s.housing_units_moe,
  s.households,
  s.households_moe,
  s.daytime_population,
  
  -- socioeconomic status (green)
  s.below_150_poverty,
  s.below_150_poverty_moe,
  s.below_150_poverty_pct,
  s.below_150_poverty_pct_moe,
  s.below_150_poverty_percentile,
  s.below_150_poverty_is_90_percentile,
  s.adults_unemployed,
  s.adults_unemployed_moe,
  s.unemployment_rate,
  s.unemployment_rate_moe,
  s.unemployment_percentile,
  s.unemployment_is_90_percentile,
  s.housing_cost_burdened,
  s.housing_cost_burdened_moe,
  s.housing_cost_burdened_pct,
  s.housing_cost_burdened_pct_moe,
  s.housing_cost_burdened_percentile,
  s.housing_cost_burdened_is_90_percentile,
  s.no_hs_diploma,
  s.no_hs_diploma_moe,
  s.no_hs_diploma_pct,
  s.no_hs_diploma_pct_moe,
  s.no_hs_diploma_percentile,
  s.no_hs_diploma_is_90_percentile,
  s.uninsured,
  s.uninsured_moe,
  s.uninsured_pct,
  s.uninsured_pct_moe,
  s.uninsured_percentile,
  s.uninsured_is_90_percentile,
  s.socioeconomic_status_sum,
  s.socioeconomic_status_percentile_rank,
  s.socioeconomic_status_90_percentile_count,
  
  -- household characteristics (orange)
  s.aged_65_over,
  s.aged_65_over_moe,
  s.aged_65_over_pct,
  s.aged_65_over_pct_moe,
  s.aged_65_over_percentile,
  s.aged_65_over_is_90_percentile,
  s.aged_17_under,
  s.aged_17_under_moe,
  s.aged_17_under_pct,
  s.aged_17_under_pct_moe,
  s.aged_17_under_percentile,
  s.aged_17_under_is_90_percentile,
  s.disabled,
  s.disabled_moe,
  s.disabled_pct,
  s.disabled_pct_moe,
  s.disabled_percentile,
  s.disabled_is_90_percentile,
  s.single_parent_hh,
  s.single_parent_hh_moe,
  s.single_parent_hh_pct,
  s.single_parent_hh_pct_moe,
  s.single_parent_hh_percentile,
  s.single_parent_hh_is_90_percentile,
  s.limited_english,
  s.limited_english_moe,
  s.limited_english_pct,
  s.limited_english_pct_moe,
  s.limited_english_percentile,
  s.limited_english_is_90_percentile,
  s.hh_characteristics_sum,
  s.hh_characteristics_percentile_rank,
  s.hh_characteristics_90_percentile_count,
  
  -- racial & ethnic minority status (purple)
  s.minorities,
  s.minorities_moe,
  s.minorities_pct,
  s.minorities_pct_moe,
  s.minorities_percentile,
  s.minorities_is_90_percentile,
  s.minority_status_sum,
  s.minority_status_percentile_rank,
  s.minority_status_90_percentile_count,
  s.african_american,
  s.african_american_moe,
  s.african_american_pct,
  s.african_american_pct_moe,
  s.hispanic,
  s.hispanic_moe,
  s.hispanic_pct,
  s.hispanic_pct_moe,
  s.asian,
  s.asian_moe,
  s.asian_pct,
  s.asian_pct_moe,
  s.american_indian,
  s.american_indian_moe,
  s.american_indian_pct,
  s.american_indian_pct_moe,
  s.native_hawaiian,
  s.native_hawaiian_moe,
  s.native_hawaiian_pct,
  s.native_hawaiian_pct_moe,
  s.two_or_more_races,
  s.two_or_more_races_moe,
  s.two_or_more_races_pct,
  s.two_or_more_races_pct_moe,
  s.other_race,
  s.other_race_moe,
  s.other_race_pct,
  s.other_race_pct_moe,
  
  -- housing type/transportation (blue)
  s.apartments,
  s.apartments_moe,
  s.apartments_pct,
  s.apartments_pct_moe,
  s.apartments_percentile,
  s.apartments_is_90_percentile,
  s.mobile_homes,
  s.mobile_homes_moe,
  s.mobile_homes_pct,
  s.mobile_homes_pct_moe,
  s.mobile_homes_percentile,
  s.mobile_homes_is_90_percentile,
  s.crowded_housing,
  s.crowded_housing_moe,
  s.crowded_housing_pct,
  s.crowded_housing_pct_moe,
  s.crowded_housing_percentile,
  s.crowded_housing_is_90_percentile,
  s.no_vehicle,
  s.no_vehicle_moe,
  s.no_vehicle_pct,
  s.no_vehicle_pct_moe,
  s.no_vehicle_percentile,
  s.no_vehicle_is_90_percentile,
  s.group_quarters,
  s.group_quarters_moe,
  s.group_quarters_pct,
  s.group_quarters_pct_moe,
  s.group_quarters_percentile,
  s.group_quarters_is_90_percentile,
  s.housing_status_sum,
  s.housing_status_percentile_rank,
  s.housing_status_90_percentile_count,
  s.hh_without_internet,
  s.hh_without_internet_moe,
  s.hh_without_internet_pct,
  s.hh_without_internet_pct_moe,
  
  s.series_themes_sum,
  s.overall_percentile_ranking,
  s.theme_90_percentile_count,
  
  r.facility_id is not null as has_readmissions_data,
  p.facility_id is not null as has_payments_data,
  h.facility_id is not null as has_hospital_data,
  s.state is not null as has_sdoh_data,
  m.facility_id is not null as has_patient_mix_data
from readmissions r
full outer join payments p
  on r.facility_id = p.facility_id
  and replace(substring(r.measure_name, 10), '-HRRP', '') = substring(replace(p.payment_measure_id, '_', '-'), 9)
left join hospitals h 
  on coalesce(r.facility_id, p.facility_id) = h.facility_id
left join footnotes f_h_rating
  on h.hospital_overall_rating_footnote = f_h_rating.footnote_id
left join footnotes f_h_mort
  on h.mort_group_footnote = f_h_mort.footnote_id
left join footnotes f_h_safety
  on h.mort_group_footnote = f_h_safety.footnote_id
left join footnotes f_h_readm
  on h.mort_group_footnote = f_h_readm.footnote_id
left join footnotes f_h_pt_exp
  on h.mort_group_footnote = f_h_pt_exp.footnote_id
left join footnotes f_h_te
  on h.mort_group_footnote = f_h_te.footnote_id
left join footnotes f_p
  on p.payment_footnote = f_p.footnote_id
left join footnotes f_p_vc
  on p.value_of_care_footnote = f_p_vc.footnote_id
left join sdoh s
  on upper(coalesce(h.county_name, p.county_name)) = upper(s.county)
  and coalesce(r.state, h.state, p.state) = s.state_abbr
left join pmr_agg m
  on coalesce(r.facility_id, p.facility_id, h.facility_id) = m.facility_id
  
where coalesce(r.state, h.state, p.state) not in ('GU', 'PR', 'VI', 'MP', 'AS')


"""

obt = pysqldf(q)
obt
obt.to_csv(r'obt.csv', index=False)

In [274]:
test1 = """
select state, max(has_readmissions_data), max(has_payments_data), max(has_sdoh_data)
from obt
group by 1
"""

test2 = """
select distinct number_of_readmissions from obt
"""


test3 = """
select distinct facility_id, facility_name,
  state,
  county_name,
  has_readmissions_data,
  has_payments_data,
  has_hospital_data,
  has_sdoh_data,
  has_patient_mix_data
from obt
where not has_patient_mix_data and condition not in ('COPD', 'CABG') and has_payments_data and has_readmissions_data
"""

test = pysqldf(test2)
test

,number_of_readmissions
0,None
1,38
2,46
3,71
4,178
...,...
278,328
279,400
280,194
281,299


In [95]:
query = """
select 
payment_measure_id is not null as has_payment_data,
measure_name is not null as has_readmission_data,
coalesce(measure_test, payment_test) as condition,
sum(count)
from test
group by 1,2,3
order by 4
"""

results = pysqldf(query)
results

,has_payment_data,has_readmission_data,condition,sum(count)
0,0,1,AMI,10
1,0,1,HF,10
2,0,1,HIP-KNEE,10
3,0,1,PN,10
4,1,0,AMI,1518
5,1,0,HF,1518
6,1,0,HIP-KNEE,1518
7,1,0,PN,1518
8,1,1,AMI,3155
9,1,1,HF,3155
